In [ ]:
from __future__ import unicode_literals
import os
from IPython.core.display import display, HTML
from IPython.display import display, Javascript, clear_output
from IPython.core.debugger import set_trace

!pip install allensdk
clear_output()
!pip install joblib
clear_output()
!pip install --upgrade "hbp-service-client==1.0.0"
clear_output()
#!conda install -y -c r rpy2 
#!conda install -c r r-base -y
clear_output()
!pip install --upgrade sklearn
clear_output()

In [ ]:
import sys
import os
import numpy as np
import scipy as sci
import csv
import pickle as pk
import h5py
import math as mt
import matplotlib.pyplot as plt
import matplotlib.axis as axis
import matplotlib
import matplotlib.legend as lgd
import sklearn.cluster as cluster
import glob
import six
import time as time
import json
import pandas as pd
import mcmodels.core as core
import mcmodels.models as models
import sklearn.decomposition as decomp
import sklearn.feature_selection as fs
import sklearn.linear_model as lm
import sklearn.metrics as metrics
import zipfile
import cortical_map_10 as cm
import nrrd
import rpy2.robjects as ro
import rpy2
import nibabel as nib
import base64
import nrrd
import skimage as ski
import imp 
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
from sklearn import multioutput
from pandas.tools.plotting import table
from itertools import groupby
from sklearn.preprocessing import Imputer, normalize, StandardScaler, scale
from scipy.stats import boxcox, mstats, normaltest, pearsonr, skew, iqr
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict,\
                                    GridSearchCV, KFold,\
                                    permutation_test_score, \
                                    StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import RandomizedLogisticRegression,\
                                 LogisticRegression,SGDClassifier, Ridge, Lars, ElasticNet
from sklearn.neural_network import MLPRegressor
from sets import Set
from sklearn import utils
from collections import OrderedDict
from joblib import Parallel, delayed
from scipy.ndimage.filters import laplace
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache 
from allensdk.api.queries.grid_data_api import GridDataApi
from scipy.ndimage.filters import laplace
from allensdk.api.queries.ontologies_api import OntologiesApi
from json import dumps as json_encode
from base64 import b64encode
from mcmodels.models.voxel import RegionalizedModel
from subprocess import call

os.environ['KMP_DUPLICATE_LIB_OK']='True'

Initially a series of data structures, files and functions that have to be utilized throughout the analysis have to be stored in the current directory space.  
The code bellow is responsible for downloading the aforementioned data from the storage space.

In [1]:
import os
from IPython.display import clear_output
client      = get_bbp_client().document
collab_path = get_collab_storage_path()
ccp_path    = 'CCP v2'
itemlist = client.listdir(os.path.join(collab_path,ccp_path))
for element in itemlist:
    if os.path.isfile('./' + str(element)) == True: continue
    if 'Library.py' in element: continue
    print 'element {} has been downloaded'.format(element)
    local_path = './' + element
    client.download_file(os.path.join(collab_path,ccp_path, element),local_path)
    clear_output()

In [2]:
from cfg import *

The functions and classes shown bellow (find_notebook, NotebookLoader, NotebookFinder) were taken from a tutorial on how to import external jupyter notebooks as modules.
Documentations about the aforementioned functions and classes can be found in this link:  
https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]   
    
sys.meta_path.append(NotebookFinder())    

In this step a series of libraries whose classes and functions have to be utilized throughout the analysis,  
are being downloaded from the storage space and imported as modules.

In [3]:
elements = ['PrimaryLibrary.ipynb','SecondaryLibrary.ipynb','ParserLibrary.ipynb']
for element in elements:
    local_path = './' + element
    client.download_file(os.path.join(collab_path, element),local_path)

import PrimaryLibrary as PL
imp.reload(PL)

import SecondaryLibrary as SL
imp.reload(SL)

import ParserLibrary as PrL
#imp.reload(PrL)

clear_output()

importing Jupyter notebook from PrimaryLibrary.ipynb
importing Jupyter notebook from SecondaryLibrary.ipynb
importing Jupyter notebook from PrimaryLibrary.ipynb
importing Jupyter notebook from SecondaryLibrary.ipynb
importing Jupyter notebook from ParserLibrary.ipynb


IOError: Unable to open file (file signature not found)

## Parameter Initialization

Initializing the parameters of the supervised learning models to be used.  
The models applied in this analysis are: 
*   classification = Random Forest classifier, Logistic Regression classifier
*   regression     = Random Forest regressor,  Ridge regressor

In [ ]:
tol_num = 0.001

logreg_params = {'penalty': ['l2'], 'C' : np.logspace(-4, 1,base = 10), \
            'solver': ['newton-cg','lbfgs','sag','liblinear','saga'], \
            'class_weight' : ['balanced']}

rfc_params = {'max_depth': [10, 60, 100, None],
             'max_features': ['auto', 'sqrt'],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2, 5, 10],
             'n_estimators': [25,50,100,200],\
             'criterion': ['gini'],
             'oob_score': [True], 'random_state' : [None],\
             'max_features' : ['auto'], 'class_weight' : ['balanced']}

ridge_params = {'alpha': [10e-4,10e-3,10e-2,0.5,1,2.5,5,7.5,10,100],
              'solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'saga'],
              'tol': [tol_num], 'max_iter' : [1000]}


sv_models = [GridSearchCV(LogisticRegression(), param_grid = logreg_params, scoring = 'f1', cv = 3, n_jobs = -1),
             GridSearchCV(RandomForestClassifier(), param_grid = rfc_params, scoring = 'f1', cv = 3, n_jobs = -1),
             DummyClassifier(),
             GridSearchCV(Ridge(), param_grid = ridge_params, 
                          scoring = 'neg_mean_squared_error', 
                          cv = 3, n_jobs = -1),
             RandomForestRegressor(n_estimators = 100),
             DummyRegressor()
            ]

Loading a series of data structures that will be used in the analysis.  
The data are being loaded for reasons of time efficiency.  
The Classes_and_Modules notebook provides comments and descriptions for the functions in which the data were created

In [ ]:
leaves         = pk.load(open('leaf_nodes.pkl','rb'))
GeneExp        = h5py.File('G_Exp.hdf5', 'r')['dataset1']
GeneMeta       = pk.load(open('GeneMeta.pkl','r'))
GeneAcros      = np.asarray([val['genes'][0]['entrez_id'] for val in GeneMeta])
GeneNames      = np.asarray([val['genes'][0]['acronym'] for val in GeneMeta])
GeneMetaDict   = {val : GeneNames[idx] for idx,val in enumerate(GeneAcros)}
ConDict        = pk.load(open('CreLineDict.pkl','rb'))
folder_name    = 'Paper4/'
targetprofiles = OrderedDict()

Storage of information regarding the mouse brain structures.
The targetprofiles dictionary specializes in target areas 
with regards to connectivity patterns in the mouse mesoconnectome.
Its keys are: 
   *           str_acronym = acronyms of target areas/structures
   *           alt laminar profiles = laminar profiles of cortical structures
   *           father               = the brain regions in which each target belongs to      
   
   
The leaf_keys contain information about target areas that are present on the finest possible level of description.
For instance areas called VISp (primary visual area) and VISp l1 (primary visual area, layer 1) could both be found 
in the dataset, and therefore the leaf_keys are being used to filter out VISp as a non-leaf key in the structure hierarchy
since a finer description is present (VISp l1).

In [ ]:
with open('acr2parent.json', 'r') as fp:
     acr2parent = json.load(fp)
with open('structures.csv','rb') as fp:
     structure = SL.readtable(fp)
struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
targetprofiles['str_acronym'] = [val for val in structure['acronym']]
targetprofiles['alt laminar profiles'] = PL.LaminarRegistration(targetprofiles['str_acronym'])
# storing layer 1 and layer 4 related areas for a latter analysis
l4_vals = [idx for idx,val in enumerate(targetprofiles['alt laminar profiles']) if val == 'layer 4']
l1_vals = [idx for idx,val in enumerate(targetprofiles['alt laminar profiles']) if val == 'layer 1']
lam_to_idx = {val:targetprofiles['alt laminar profiles'][idx] for idx,val in enumerate(structure['id'])}

# storage of leaf-level structures
leaf_keys = [int(val[0]) for val in leaves]  

for key in targetprofiles.keys():
    targetprofiles[key] = [val for idx,val in enumerate(targetprofiles[key]) if idx in leaf_keys]
targetprofiles['father'] = []
for strut in targetprofiles['str_acronym']:
    father = [val for key,val in acr2parent.items() if key == strut]
    if father == []:
       targetprofiles['father'].append(strut) 
    else:
       targetprofiles['father'].append(father[0])  

The params dictionary stores information which will be used throughout the analysis.
It will be passed as a parameter through the functions and updated at various steps. 

In [ ]:
params = {'alter' : None, 'fetch' : 'all', 'distance prior': False,\
          'acronyms': targetprofiles['str_acronym'],\
          'method': 'classification', 'leaf': True, \
          'validation' : [StratifiedKFold(n_splits = 3, shuffle = True), KFold(n_splits = 3, shuffle = True)],\
          'prefix': 'Paper4/', 'primary models': [i for i in sv_models], \
          'Gene Acronyms Original': GeneAcros, 'Gene Metadata': GeneMetaDict,
          'leaf_keys' : leaf_keys}

Conversion of the Gene Expression dataset as a numpy array for analysis,  
initialization of various dictionaries to store preprocessed data
and results per tracer category (driver line),  
and initialization the the MesoconnectomePredictor for its
functions to be utilized

In [ ]:
GeneExp = np.asarray(GeneExp, dtype = np.float32)
ConStr_proc = {}; GeneExp_proc = {}; y = {}

# matrix reduction to leaf-level structures
GeneExp  = GeneExp[leaf_keys,:]  
MesoPred = PL.MesoconnectomePredictor(GeneExp, ConDict['wild_type']['ConMat'])

## Predictive Pipeline  
### Author: Nestor Timonidis
In our examples I utilize the unionized anatomical tract tracing data taken from the Allen Institute for Brain Science, as shown in the Class_and_Modules notebook. The data come in the form of a dictionary with nested matrices, where each dictionary corresponds to a tracer category like wild type or Syt6-Cre_KI148 cre-line experiments for instance.   
In order to demonstrate the MesoPred tool, a number of categories are selected as shown bellow. This analysis can take more than a day if run for all possible tracer categories. Therefore, for reasons of time efficiency one could skip this cell and move to the next one where the results can be loaded by stored pickle files.


In this pipeline a number of steps is being followed:  
0.  Selection of the tracer category (driver) of interest, reduction of tracing data to the leaf-level structures and storage of a number of parameters to be used in the analysis, such as the layer profiles and abbreviations of the target areas of the dataset.
1.  Preprocessing of the gene expression and tract tracing data. The function called in that step is PreProcessing
2.  Prediction with a logistic or linear ridge regression model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
3.  Prediction with a random forest classifier or regressor model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
4.  Prediction with a control (baseline) model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
5.  Storage of the predicted connectivity patterns, probability scores and gene importance (coefficient) scores per tracer in dictionaries stratified by the tested driver categories. The function called in that step is UnravelResults.
6.  Based on the stored connectivity patterns and the actual (ground truth) ones, evaluation of the predictive process with series of statistical measures. Examples include r squared and root mean squared error for regression, and f1-score and area under the roc curve for classification. The function called in that step is Evaluation.
8. Estimate the stability of the fittest model based on the optimal hyperparameters selected for each fold of the CustomCrossval function (step 3). In case that the model is considered as stable, then the model fits the whole data with the optimal hyperparameters

In [ ]:
for key in ConDict.keys():
     
    ## Step 0:
    # measure used not to repeat predictions that have already been done
    outfile = key + '_all_results' 
    if os.path.isfile('./' + outfile + '_ridge.pkl') == True: continue 
    ConStr  = np.asarray(ConDict[key]['ConMat'], dtype = np.float32)
    ConStr = ConStr[leaf_keys,:]
    params['injection_number'] = np.shape(ConStr)[1]  
    params['cre-line'] = key
    params['structure-abbrev'] = ConDict[key]['structure-abbrev']
    params['layer'] = ConDict[key]['layer'][0]
    params['cell-type'] = ConDict[key]['cell-type'][0]
    params['method'] = 'regression'
 
    print 'Driver line: '+ str(key) + '\nTracer number: ' + str(ConStr.shape[1]) 
    
    ## Step 1:
    GeneExp_proc[key], ConStr_proc[key], y[key], sc_sqrt = MesoPred.PreProcessing(
                                                    GeneExp, ConStr, params)
    str_copy = np.delete(targetprofiles['str_acronym'],params['nanCons'],0); str_copy = np.delete(str_copy,params['nanGenes'],0)
  
    start = time.time()
    ## Step 2:
    print 'Ridge ' + params['method'] +  ' predictive phase ..'    
    ridge_pred  = list(SL.CustomCrossval(params['primary models'][3], GeneExp_proc[key], 
                                         ConStr_proc[key], params['validation'][1]))
    ridge_pred.append(GeneExp_proc[key])
    #pk.dump(ridge_pred, open(params['cre-line'] + '_ridge_pred.pkl', 'wb'))
    print 'Ridge ' + params['method'] +  ' has been completed' 
    
    ## Step 3:
    print 'Random forest ' + params['method'] + ' predictive phase ..'
    rf_pred  = list(SL.CustomCrossval(params['primary models'][4], GeneExp_proc[key], 
                                      ConStr_proc[key], params['validation'][1]))
    rf_pred.append(GeneExp_proc[key])                                  
    #pk.dump(rf_pred, open(params['cre-line'] + '_rf_pred.pkl','wb'))
    print 'Random forest ' + params['method'] +  ' has been completed' 
    
    ## Step 4:
    print 'Control model ' + params['method'] +  ' predictive phase ..'
    bl_pred  = list(SL.CustomCrossval(params['primary models'][5], GeneExp_proc[key], 
                                      ConStr_proc[key], params['validation'][1]))
    bl_pred.append(GeneExp_proc[key])                                       
   
    #pk.dump(bl_pred, open(params['cre-line'] + '_bl_pred.pkl','wb'))
    print 'Control ' + params['method'] +  ' has been completed'  
    
    end = time.time() 
    print  'Elapsed time: ' + str((end - start)/60)   
    
    ## Step 5:
    print 'Unraveling of results phase ..' 
    ClfResults_ridge = MesoPred.UnravelResults(ridge_pred, params, sc_sqrt)
    ClfResults_rf = MesoPred.UnravelResults(rf_pred, params, sc_sqrt)
    #pk.dump(ClfResults_rf, open(outfile + '_rf.pkl','wb'))
    ClfResults_bl = MesoPred.UnravelResults(bl_pred, params, sc_sqrt)
    #pk.dump(ClfResults_bl, open(outfile + '_bl.pkl','wb')) 
     
    ## Step 6:
    print 'Evaluation of results phase ..'
    MesoPred.Evaluation(ClfResults_ridge, ClfResults_rf, ClfResults_bl, params) 
    
    # Step 7:
    unanimity_thr = 0.2
    verdict, params_to_fit = MesoPred.StabilityInvestigator(ClfResults_ridge['Model Storage'], unanimity_thr)
    if verdict == 'Full Fit': 
        final_model = Ridge(**params_to_fit).fit(ridge_pred[5], ridge_pred[2])
    elif verdict == 'Partial Fit': 
        final_model = [Ridge(**model.best_params_).fit(ridge_pred[5], ridge_pred[2])\
                       for model in ClfResults_ridge['Model Storage']]
    
    ClfResults_ridge['final model'] = final_model
    final_model = []
    pk.dump(ClfResults_ridge, open(outfile + '_ridge.pkl','wb'))
    
    clear_output()

In case that a set of results from a number of tracer categories have been stored,  
they can be loaded as shown bellow in order to save time.   
The data from the ridge regression model are being used   
due to the model's superior performance over the rest of  
the models as shown by the cross-validation evaluation method.

In [ ]:
ClfResults_ridge = {}; ridge_pred = {}
unanimity_thr = 0.2
for key in ConDict.keys():
    if os.path.isfile('./' + key + '_all_results' + '_ridge.pkl') == True:
        ClfResults_ridge[key] = {key:val for key,val in tmp2.items() if key != 'Model Storage'}
        ridge_pred[key] = [ClfResults_ridge[key]['y_actual'],
                           ClfResults_ridge[key]['y_pred'],
                           ClfResults_ridge[key]['Gene Expression']]
        set_trace()
        '''verdict, params_to_fit = MesoPred.StabilityInvestigator(tmp2['Model Storage'], unanimity_thr)
        if verdict == 'Full Fit': 
            final_model = 1 #Ridge(**params_to_fit).fit(ridge_pred[key][2], ridge_pred[key][1])
        elif verdict == 'Partial Fit': 
            final_model = 2 #[Ridge(**model.best_params_).fit(ridge_pred[key][2], ridge_pred[key][1])\
                           #for model in tmp2['Model Storage']]'''
        tmp2 = []; final_model = []

One of the last demonstrations regarding the predictive pipeline is about binarized connectivity patterns. This could be useful in the case that a particular researcher is not interested in the connectivity strength between two brain areas but in the presence of connections between areas, whose strength exceeds a certain threshold. For that reason, the Convert2ROC function is being called for all predicted connectivity patterns.     
The idea is the utilize the Area under the ROC curve (AURoc) as an evaluation measure in order to estimate the most optimal binarizing threshold. The threshold for which the AURoc is being maximized is selected as the most optimal. 
The steps that describe this procedure are being explained in the Convert2ROC function.

In [ ]:
#MesoPred = MesoconnectomePredictor(GeneExp, ConDict['wild_type']['ConMat'])
cutoff_col = {}
for driver_line in ridge_pred.keys():
    print 'driver line: ' + str(driver_line)
    cutoff_col[driver_line] = MesoPred.Convert2ROC(ridge_pred[driver_line][1],ridge_pred[driver_line][0])

Lets take the case where we want to binarize the connectivity patterns in experiments belonging to the cre-line tracers tested in our analysis such as Cre_GN220. We utilize the our results from the previous function as shown bellow 

In [ ]:
experiments_len = np.shape(ridge_pred['Ntsr1-Cre_GN220'][0])[1]
BinarizedConnections = {}
for driver_line in ridge_pred.keys():
    BinarizedConnections[driver_line] = np.zeros(np.shape(ridge_pred['Ntsr1-Cre_GN220'][0]))
    for tracer_num in range(experiments_len):  
        BinarizedConnections[driver_line][:,tracer_num] = SL.BinarizeTheVector\
                                                         (ridge_pred['Ntsr1-Cre_GN220'][0][:,tracer_num],\
                                                            cutoff_col['Ntsr1-Cre_GN220'][tracer_num][0])

## VISUALIZATION CHAPTER  
### Authors: Nestor Timonidis, Paul Tiesinga
In this section, a number of steps are being followed   
for visualizing brain slices containing gene expression   
or connectivity information


Utilization of the MouseConnectivityCache (mcc) tool, provided by the Allen Institute for Brain Science Software Development Toolkit (SDK) at:  
https://alleninstitute.github.io/AllenSDK/_modules/allensdk/core/mouse_connectivity_cache.html    
The cortical flatmap templates used in this analysis, were downloaded from the informatics-archive of the Allen Institute for Brain Science:   
http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/cortical_coordinates/ccf_2016/  
The cortical_map_10.py script is a slightly modified version of the cortical_map.py script that was provided by the Allen Institute's github as part of the mouse_connectivity_models tool:   
https://github.com/AllenInstitute/mouse_connectivity_models  
The difference with the original script is that cortical_map_10 is modified to incorprorate cortical flatmap templates of 10 micrometer resolution.  
With the mcc tool, the annotation volume of the mouse brain is being downloaded in both 25 and 10 micrometer resolution for visualization purposes.

In [ ]:
mcc_25 = MouseConnectivityCache(resolution = 25)
mcc_10 = MouseConnectivityCache(resolution = 10)
Annotation_25, annot_info = mcc_25.get_annotation_volume()
Annotation_10, annot_info = mcc_10.get_annotation_volume()

### Application of a set of experiment ids in the brain visualization tools (Brain Plotter)  

In order to make a cortical flatmap visualization for the mouse cortex,    
we need to initially ensure that we have reduced the complexity  
of structural brain areas to a layer inspecific level that corresponds    
to clear anatomically distinct areas like the primary visual or motor cortex.   
Therefore we utilize the MouseConnectivityCache tool to get the ontology    
tree of brain areas, then we reduce it by the isocortex and for each    
isocortical area, we take the identity of its father node, or an area  
belonging up in the tree hierarchy.      
Having achieved that, the next step is to replace the identity of each    
area in the annotation volume structure by the father identity.  

In [ ]:
# grab the StructureTree instance
structure_tree = mcc_10.get_structure_tree()
oapi           = OntologiesApi()
# Get isocortex summary structures
kid_to_father = {}
summary_structures = structure_tree.get_structures_by_set_id([688152357])
for val in summary_structures:
    kids = structure_tree.descendant_ids([val['id']])
    for kid in kids[0]:
        kid_to_father[kid] = val['id']

# For each distinct isocortical area, replace its children annotation with the area's id
father_acros_indices = {val['acronym']:val['id'] for val in summary_structures}

#store the original annotation shape because it will be flatten for computational simplicity when updating the parent nodes
orig_annot_shape = np.shape(Annotation_10)
flat_annot = np.ndarray.flatten(Annotation_10, order = 'C')

# Create a copy of the annotation structure that reduces areas to their parent nodes
Annotation_copy = np.zeros(np.shape(flat_annot))

for kid_id,parent_id in kid_to_father.items():
    tmp  = flat_annot == int(kid_id)
    nzero_ids = np.nonzero(tmp)
    Annotation_copy[nzero_ids] = parent_id 
     
with h5py.File('Annotation_copy.hdf5','w') as f2:
     f2.create_dataset('dataset1', data = Annotation_copy)

Selection of the 11th and 21st experiment for demonstration of the BrainPlotter tool.   
Data structures for storing the mapping of unionized projections to volumetric scale,  
for the actual and predicted patterns, are being initialized.  
Moreover, the dictionary that maps the identities of structures to their positions  
in the original raw unionized dataset is being updated to include only the   
structures that were used in the predictive pipeline.  

In [ ]:
tracer_for_selection = 'Ntsr1-Cre_GN220'
experiment_to_vis = [10,20]
UnionToVoxel_pred_25 = {}; UnionToVoxel_pred_10 = {}; UnionToVoxel_25 = {}; UnionToVoxel_10 = {}
struct_idx_dict_cp = struct_idx_dict.copy()
for key,value in struct_idx_dict.items():
    if value not in params['remaining_indices']:
       del struct_idx_dict_cp[key]
struct_idx_dict = struct_idx_dict_cp    
#Annotation_copy = np.asarray(h5py.File('Annotation_copy.hdf5','r')['dataset1'], dtype = np.int32)

For each one of the selected experiments:  
0.   Initializing the data for usage during the visualization process.
1.   Mapping of unionized projections to volumetric space for actual and predicted projection patterns,   using the FromUnion2Voxel function.
2.   Repetition of step 1 for 10 micrometers instead of 25, since the cortical flatmap visualization requires 10 micrometer resolution, while the brain slice visualization requires 25 micrometers.
3.   Initialization of the BrainPlotter class instances,
4.   Call of the plot_slice function to visualizing projections of the currently selected experiment in the superior-inferior and left-right coordinates using a brain slice.
5.   Call of the plot_flatmap function to visualizing projections of the currently selected experiment on the mouse cortex, using a cortical flatmap.

In [ ]:
for j,experiment in enumerate(experiment_to_vis):
    
    # Step 0
    key_to_save = tracer_for_selection + '_' + str(j)
    exp_id = ConDict[tracer_for_selection]['id'][j]
    ConStr = ConDict[tracer_for_selection]['ConMat']
    Pred_ConStr = ridge_pred[tracer_for_selection]
    
    # Step 1
    UnionToVoxel_25[key_to_save], UnionToVoxel_pred_25[key_to_save] = SL.FromUnion2Voxel(
         Annotation_25, struct_idx_dict, Pred_ConStr[1][:,j], params, 
         Pred_ConStr[0][:,j])
    with h5py.File('./UnionToVoxel_25_for_' + str(key_to_save) + '.hdf5','w') as fp:
        fp.create_dataset('dataset1', data = UnionToVoxel_25[key_to_save], 
                          dtype = np.float32) 
    with h5py.File('./UnionToVoxel_25_pred_for_' + str(key_to_save) + '.hdf5','w') as fp:
        fp.create_dataset('dataset1', data = UnionToVoxel_pred_25[key_to_save], dtype = np.float32)  
    
    # Step 2   : Due to limitations on available memory, we shall skip the volumetric mapping on the 10 micrometer 
    #            resolution. We shall instead show in the next cell the 10 micrometer cortical flatmap on the 
    #            actual raw volumetric data
    
    # UnionToVoxel_10[key_to_save], UnionToVoxel_pred_10[key_to_save] = \
    #                  FromUnion2Voxel(Annotation_10, struct_idx_dict,\ 
                                       #Pred_ConStr[1][:,j], Pred_ConStr[0][:,j])    
        
    # Step 3   
    #BP_10 = BrainPlotter(mcc_10, experiment_id = exp_id, res = 10, 
                          #driver = tracer_for_selection + str(exp_id))
    BP_25 = SL.BrainPlotter(mcc_25, experiment_id = exp_id, 
                            res = 25, driver = tracer_for_selection + str(exp_id))
    
    # Step 4
    BP_25.plot_slice(struct_idx_dict, 'None', Annotation_25, 
                     matrix = UnionToVoxel_25[key_to_save],\
                     savefile = 'slice visualization for : ' + 
                     str(tracer_for_selection) + ' ' +  str(j+1) + '.png')
    BP_25.plot_slice(struct_idx_dict, 'None', Annotation_25, 
                     matrix = UnionToVoxel_pred_25[key_to_save],\
                     savefile = 'slice visualization for : ' + 
                     str(tracer_for_selection) + ' ' +  str(j+1) + ' (predicted).png')

    # Step 5:  Skipping the step (see Step 2 comments)
    # BP_10.plot_flatmap('None', Annotation_copy, matrix = UnionToVoxel_10[key_to_save],\
    #                   savefile = 'flatmap visualization for : ' 
                        #+ str(tracer_for_selection) + ' ' +  str(j+1) + '.jpg')
    # BP_10.plot_flatmap('None', Annotation_copy, matrix = UnionToVoxel_pred_10[key_to_save],\
     #                  savefile = 'flatmap visualization for : ' + str(tracer_for_selection) 
                        #+ ' ' +  str(j+1) + '.jpg') 

We are also interested in producing the aforementioned figures with the actual raw volumetric data. Therefore, steps 3-5 are repeated without giving any volumetric matrix as input to the plot_slice and plot_flatmap functions. In that case, the identities of the original experiments are used to download and visualize the actual raw data.

In [ ]:
for j,experiment in enumerate(experiment_to_vis):
    
    # Step 0
    exp_id = ConDict[tracer_for_selection]['id'][j]

    # Step 3          
    BP_10 = SL.BrainPlotter(mcc_10, experiment_id = exp_id, res = 10, 
                            driver = tracer_for_selection + str(exp_id))
    BP_25 = SL.BrainPlotter(mcc_25, experiment_id = exp_id, res = 25, 
                            driver = tracer_for_selection + str(exp_id))
    
    # Step 4
    BP_25.plot_slice(struct_idx_dict, 'None', Annotation_25, matrix = 'None',\
                     savefile = 'slice visualization for : ' + 
                     str(tracer_for_selection) + ' ' +  str(j+1) + '.png')
    # Step 5
    BP_10.plot_flatmap('None', Annotation_10, matrix = 'None',\
                       savefile = 'flatmap visualization for : ' + 
                       str(tracer_for_selection) + ' ' +  str(j+1) + '.jpg')
    

## Link to Scalable Brain Atlas Composer Section
### Authors: Rembrandt Bakker, Nestor Timonidis

In this part, the Scalable Brain Atlas (SBA) Composer visualization tool is being used to visualize actual and predicted data that have been used and processed throughout the analysis.      
The website of the SBA Composer can be accessed by this link:  
https://scalablebrainatlas.incf.org/composer/?template=ABA_v3  
A series of steps bellow describe the visualization process.

In order for results to be visualized, a conversion to json format is necessary.
Therefore the dictionary bellow indicates the format that results must have in order to be visualized.
Specifically:  
*    provider    = the name of the tool to be used. sba stands for scalable brain atlas.
*    atlas       = the Common Coordinate Framework (CCF) version that was used by the Allen Institute to process the raw data that the user has used throughout the analysis. In this case ABA_v3 stands for Allen Brain Atlas version 3, since our data were processed according to that framework.  
*    orientation = the orientation used in 3D space for the input. RAS stands for: right-anterior-superior.
*    unit        = the scaling unit. mm stands for milimiters.


In [ ]:
JSonDict = {'bas': {'provider': "sba",
             'atlas': "ABA_v3",
             'orientation': "RAS",
             'unit': "mm"},
             'style': {}, 'markers':[]} 

According to the aforementioned format, we select predictions of all tracing data tested by our predictive model.  
For non wild-type, meaning cre-line experiments, we also incorporate information regarding the layer and cell types  
most commonly expressed in the injected source brain areas.  
Furthermore, we select to visualize the r2 measure for evaluation of predictive performance.   
For each individual experiment we embed the r2 measure in the spatial locations of the structural  
brain area corresponding to the injected source location.

In [ ]:
for tracer_name in ridge_pred.keys():
    if tracer_name == 'wild_type':
       tracer_layer = ''
       tracer_cell  = ''
    else:
       tracer_layer = ConDict[tracer_name]['layer'][0]
       tracer_cell =  ConDict[tracer_name]['cell-type'][0]
    for idx, measure in enumerate(ClfResults_ridge[tracer_name]['r2']):
        JSonDict['markers'].append({})
        JSonDict['markers'][len(JSonDict['markers'])-1]['value']    = str(measure[0])
        JSonDict['markers'][len(JSonDict['markers'])-1]['position'] = [str(coo) for coo in ConDict[tracer_name]['Coordinates'][idx]]
        JSonDict['markers'][len(JSonDict['markers'])-1]['region']   = ConDict[tracer_name]['structure-abbrev'][idx]
        JSonDict['markers'][len(JSonDict['markers'])-1]['relsize']  = "0.5"
        JSonDict['markers'][len(JSonDict['markers'])-1]['cre-line'] = tracer_name
        if 'layer' in ConDict[tracer_name]:
            JSonDict['markers'][len(JSonDict['markers'])-1]['layer']  = ConDict[tracer_name]['layer'][idx] 
 

In this step, a sbaInterface_class object is being initialized to be used in the visualization process.    
The sbaInterface_class acts as the interface between our data and the tool.  
As an initial test, a window with the SBA composer opens without data.  

In [ ]:
from IPython.display import display, Javascript, clear_output

# SBA Composer page to load: here it is the -dev version and atlas ABA_v3
sbaHost = 'https://scalablebrainatlas.incf.org/composer-dev';
sbaUrl = sbaHost+'/?template=ABA_v3';

class sbaInterface_class:
    def __init__(this,sbaUrl):
        # In Javascript, load sbaInterface.js and create a new sbaInterface object
        display(Javascript("""
            var script = document.createElement('script');
            script.src = '{}/../js/sba-interface.js';
            script.onload = function() {{
              window.global_sbaInterface = new sbaInterface_class('{}');
            }}
            document.head.appendChild(script)
        """.format(sbaHost,sbaUrl)))
        clear_output()
        
    def send(this,sbaCommand):
        display(Javascript("""window.global_sbaInterface.send({})""".format(json_encode(sbaCommand))))
        clear_output() # prevents clogging the Jupyter notebook

sbaInterface = sbaInterface_class(sbaUrl)


In this step, the predictive results in json format  
are being given as input to the send function of the    
sbaInterface object. The result is visualization of   
our selected measure across the mouse brain.  

In [ ]:
#sbaInterface.send(json.dumps({'method':'Composer.scatter3d' ,'params' : JSonDict}))
sbaCommand = {
    "method":"Composer.scatter3d",
    'params' : JSonDict
}
sbaInterface.send(sbaCommand)

After the prediction of connectivity patterns with the training of unionized data has been evaluated   
and visualized,the next and final step is the visual inspection of the connectivity patterns in the   
volumetric space of 25 or 10 micrometers. In this example, the UnionToVoxel_10 is being used,   
that in previous steps stored a mapping of the actual and predicted connectivity patterns   
in the volumetric space. For each individual experiment whose actual and predicted patterns   
have been stored by UnionToVoxel_10, the data structure is being converted into nifti format  
and given as input to the send function in order to be visualized.  
Following that command, a new window will pop-up and the user can  visually inspect the patterns  
by zooming in/out and rotating the mouse brain.

In [ ]:
nifti_urls = SL.Save2Nifti(UnionToVoxel_25, affine_scale = 0.025)
exp_to_select = nifti_urls.keys()[0]
with open(nifti_urls[nifti_urls.keys()[0]], "rb") as fp:
  sbaCommand = {
    "method":"Composer.import",
    "params": {
      "name": nifti_urls[nifti_urls.keys()[0]].split('.')[0] + '.bas{sba.ABA_v3^corner,PIR,mm}.nii.gz', 
      "encoding": 'base64',
      "contents": base64.b64encode(fp.read()).decode('utf-8')
    }
  }

# send the command to sbaComposer
sbaInterface.send(sbaCommand)

**As a side note**: The plane corresponding to the brain slice at the SBA composer is not fixed but can be rotated with the use of the left click or zoomed with the right click. Moreover, the toolbar at the left of the screen provides to the user a set of visualization configurations.
Furthermore in terms of predictive results, the black color at the SBA composer represents a lack of connection between the voxels and the corresponding target area, while the white color represents connection between them.

Clearing storage space from the nifti volumes

In [ ]:
!ls

In [ ]:
!rm *.nii.gz